<a href="https://colab.research.google.com/github/Jumabek/net_intrusion_detection/blob/transformer/transformer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1-t3RdDpmqMs4ABt9oobSapeNYTZJ9tpu
!unzip MachineLearningCSV.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-t3RdDpmqMs4ABt9oobSapeNYTZJ9tpu
To: /content/MachineLearningCSV.zip
100% 235M/235M [00:04<00:00, 47.1MB/s]
Archive:  MachineLearningCSV.zip
   creating: MachineLearningCVE/
  inflating: MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv  
  i

In [ ]:
!wget https://raw.githubusercontent.com/otabek5454/net_intrusion_detection/master/models.py
!wget https://raw.githubusercontent.com/Jumabek/net_intrusion_detection/develop/preprocessing.py

--2023-08-13 02:43:43--  https://raw.githubusercontent.com/otabek5454/net_intrusion_detection/master/models.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1551 (1.5K) [text/plain]
Saving to: ‘models.py’

models.py           100%[===================>]   1.51K  --.-KB/s    in 0s      

2023-08-13 02:43:44 (33.2 MB/s) - ‘models.py’ saved [1551/1551]

--2023-08-13 02:43:46--  https://raw.githubusercontent.com/Jumabek/net_intrusion_detection/develop/preprocessing.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3790 (3.7K) [text/

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from preprocessing import load_data, balance_data, normalize
import pandas as pd
import glob
import os
import numpy as np

In [ ]:
dataroot = 'MachineLearningCVE/'

from preprocessing import read_data
data = read_data(dataroot,'*.pcap_ISCX.csv')

# Load and preprocess the data
X, y = load_data(dataroot)

[########################################] | 100% Completed | 29.67 s
[########################################] | 100% Completed | 29.83 s
there are 2830743 flow records with 79 feature dimension
stripped column names
dropped bad columns
There are 0 nan entries
converted to numeric


In [ ]:
X, y = balance_data(X, y, seed=42)
X = normalize(X)

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and remaining sets first
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Split the remaining data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42)

# Convert the data into PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

In [ ]:
# Convert data to tensors
X_train = torch.tensor(X_train, dtype=torch.float)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create PyTorch Datasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Dataloader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

<ipython-input-7-498265b65bb0>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train, dtype=torch.float)
<ipython-input-7-498265b65bb0>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.long)
<ipython-input-7-498265b65bb0>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test, dtype=torch.float)
<ipython-input-7-498265b65bb0>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clo

In [ ]:
class CustomTransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=64, nhead=8, num_layers=4, dim_feedforward=128, dropout=0.1):
        super(CustomTransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(1)  # Add a fake batch dimension for the transformer
        x = self.transformer(x, x)  # Encoder-Decoder Self-Attention
        x = x.squeeze(1)
        x = self.fc(x)
        return x


In [ ]:
# Model Configuration
input_dim = X_train.shape[1]
num_classes = len(np.unique(y))
model = CustomTransformerModel(input_dim, num_classes)
model = model.cuda()

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dataloader with new batch size
batch_size = 5120
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# Training loop
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for i, (data, labels) in enumerate(train_loader):
        data = data.cuda()  # Move data to GPU
        labels = labels.cuda()  # Move labels to GPU

        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print('Epoch [{}/10], Loss: {:.4f}'.format(epoch + 1, running_loss / len(train_loader)))

Epoch [1/10], Loss: 0.5223
Epoch [2/10], Loss: 0.2106
Epoch [3/10], Loss: 0.1879
Epoch [4/10], Loss: 0.1587
Epoch [5/10], Loss: 0.1641
Epoch [6/10], Loss: 0.1319
Epoch [7/10], Loss: 0.1331
Epoch [8/10], Loss: 0.1353
Epoch [9/10], Loss: 0.1266
Epoch [10/10], Loss: 0.1213


Epoch [1/10], Loss: 0.4854

Epoch [2/10], Loss: 0.2209

Epoch [3/10], Loss: 0.1967

Epoch [4/10], Loss: 0.1831

Epoch [5/10], Loss: 0.1821

Epoch [6/10], Loss: 0.1861

Epoch [7/10], Loss: 0.1763

Epoch [8/10], Loss: 0.1836

Epoch [9/10], Loss: 0.1782

Epoch [10/10], Loss: 0.1675

In [ ]:
# Testing
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (data, labels) in enumerate(test_loader):
        data = data.cuda()
        labels = labels.cuda()
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 89.25487092892082 %


In [ ]:
from sklearn.metrics import classification_report

# Testing
model.eval()

all_labels = []
all_predictions = []

with torch.no_grad():
    correct = 0
    total = 0
    for i, (data, labels) in enumerate(test_loader):
        data = data.cuda()
        labels = labels.cuda()
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

accuracy = 100 * correct / total
print('Test Accuracy: {:.2f} %'.format(accuracy))

report = classification_report(all_labels, all_predictions)
print('Classification Report:')
print(report)


Test Accuracy: 89.26 %
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.88      0.85     28182
           1       0.93      0.90      0.92     28303
           2       0.97      0.96      0.97     28127
           3       0.95      0.97      0.96     28212
           4       0.93      0.97      0.95     28332
           5       0.98      0.96      0.97     28210
           6       0.95      0.98      0.96     28405
           7       0.99      0.99      0.99     28245
           8       1.00      1.00      1.00     28434
           9       0.99      0.95      0.97     28473
          10       0.98      0.95      0.97     28150
          11       0.83      0.96      0.89     28393
          12       0.60      0.47      0.53     28530
          13       0.91      0.75      0.83     28446
          14       0.58      0.68      0.63     28169

    accuracy                           0.89    424611
   macro avg       0.89      0.89 